# ChatGPT API: Few-Shot Multiclass Classification

## The Association for Computational Linguistics
## WASSA 2023 Shared Task on Multi-Label and Multi-Class Emotion Classification on Code-Mixed Text Messages
See more details [here](https://codalab.lisn.upsaclay.fr/competitions/10864#learn_the_details)

In [1]:
import openai
import numpy as np
import pandas as pd
import sklearn
import re, os
import time
import zipfile, pickle
from typing import List
from copy import deepcopy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier, RidgeClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from openai.embeddings_utils import cosine_similarity
from tqdm.autonotebook import tqdm
import random
import tiktoken
import backoff
tqdm.pandas()

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 400)
#os.path.join()

/var/folders/vb/p2r9brhx2335cwnww04p9w180000gn/T/ipykernel_39502/495498401.py:19: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    '''Return number of tokens used in a list of messages for ChatGPT'''
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        #print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo":
        #print("Warning: gpt-3.5-turbo may change over time. Returning num tokens assuming gpt-3.5-turbo-0301.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301")
    elif model == "gpt-4":
        #print("Warning: gpt-4 may change over time. Returning num tokens assuming gpt-4-0314.")
        return num_tokens_from_messages(messages, model="gpt-4-0314")
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif model == "gpt-4-0314":
        tokens_per_message = 3
        tokens_per_name = 1
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [14]:
# in the order of decreasing frequency
label2key = {
    'neutral': 0,
    'joy': 1,
    'trust': 2,
    'disgust': 3,
    'optimism': 4,
    'anticipation': 5,
    'sadness': 6,
    'fear': 7,
    'surprise': 8,
    'anger': 9,
    'pessimism': 10,
    'love':  11,
}
key2label = { v: k for k,v in label2key.items()}
key2label

{0: 'neutral',
 1: 'joy',
 2: 'trust',
 3: 'disgust',
 4: 'optimism',
 5: 'anticipation',
 6: 'sadness',
 7: 'fear',
 8: 'surprise',
 9: 'anger',
 10: 'pessimism',
 11: 'love'}

In [5]:
random_state = 47

# Load and Prepare Data

In [26]:
file1    = 'data/mcec_train_translated.pkl'
df_train = pd.read_pickle(file1)

file2    = 'data/df_dev_100_closest_GptEmbeddings.pkl'
df_dev   = pd.read_pickle(file2)

file3    = 'data/mcec_test.csv'
df_test  = pd.read_csv(file3)

file4    = 'data/sample_submission/predictions_MCEC.csv'
sample_submission = pd.read_csv(file4)

print(df_train.shape, df_dev.shape, df_test.shape, sample_submission.shape)

(9530, 5) (1191, 13) (1191, 1) (1191, 1)


In [27]:
df_train['target'] = df_train['emotion'].map( label2key )
df_dev['target']   = df_dev['emotion'].map( label2key )

In [28]:
print(df_train['emotion'].value_counts(), '\n')
print(df_train['target'].value_counts())
df_train.head()

neutral         3262
trust           1118
joy             1022
optimism         880
anticipation     832
disgust          687
sadness          486
fear             453
anger            226
surprise         199
love             187
pessimism        178
Name: emotion, dtype: int64 

0     3262
2     1118
1     1022
4      880
5      832
3      687
6      486
7      453
9      226
8      199
11     187
10     178
Name: target, dtype: int64


,text,emotion,translated_hi,translated_ur,gpt_embedding,target
0,Yes.I am in fyp lab cabin.but fyp presentation...,neutral,Yes.i am in fyp lab cabin.but fyp presentation...,Y. Um in Fap Lab Cabin. Butt Fap Presentations...,"[-0.005477939732372761, -0.01738985814154148, ...",0
1,Yar insan ka bcha bn chawliyn na mar :p,joy,"Dude become a child of a human being, do not die.",Dude human beings do not die: P: P,"[0.0006696455529890954, -0.006965265609323978,...",1
2,Terai uncle nai kahna hai kai ham nai to bahr ...,disgust,Your Uncle Nai says that we had sent out money,Your Ankali says that we sent out money and wa...,"[0.021171217784285545, -0.02109299972653389, 0...",3
3,Yr ajao I m cming in the club,neutral,YR AJAO I'M Coming in the Club,Yer organs were the club,"[-0.010511564090847969, -0.02738134376704693, ...",0
4,Mje wese Nimra ahmad ka Qur'aan ki aayaat k ba...,joy,Mje wes nimra ahmad ka qur'aan ki aayaat k bar...,Mje Wese Nimra Ahmad Ka Qur'aan Ki Aayaaat K B...,"[-0.0016743674641475081, -0.021855581551790237...",1


In [29]:
print(df_dev['emotion'].value_counts(), '\n')
print(df_dev['target'].value_counts())
df_dev.head()

neutral         388
joy             131
trust           125
disgust         113
optimism        110
anticipation     94
sadness          62
fear             52
surprise         35
anger            35
pessimism        29
love             17
Name: emotion, dtype: int64 

0     388
1     131
2     125
3     113
4     110
5      94
6      62
7      52
8      35
9      35
10     29
11     17
Name: target, dtype: int64


,text,emotion,target,gtp_translated,translated_hi,translated_ur,text_clean,gpt_pred,gpt_pred_num,gpt_translated2,gpt_translated2_corrected,gpt_embedding,closest_texts
0,Tension lene ki koi baat ni,neutral,0,There's no need to take tension.,There is nothing to take tension,Any talk of taking tangoes,Tension lene ki koi baat ni,neutral,1,There's no need to worry.,There's no need to worry.,"[-0.00021548081713262945, 0.005029499996453524...","[Tension na lo hi jaw ga, O nahi yar tension n..."
1,Main ghar punch gya hun or ab spny laga hun,neutral,0,I have reached home and now I am going to sleep.,I have gone home punch and now I am Sapni,I have gone home punch and now dreams,Main ghar punch gya hun or ab spny laga hun,neutral,1,I have reached home and now I am going to sleep.,I have reached home and now I am going to sleep.,"[-0.0010164333507418633, -0.013282055966556072...","[Main tu ghar hi rehta main kahan ja skta, Ghr..."
2,Nai mje nai mili mail..mene check ki ti,pessimism,10,"I didn't receive any mail, I had checked.",Nai Maje Nai Mile Mail .. I checked,Ni Ni Ni Mille Mail,Nai mje nai mili mail .. mene check ki ti,neutral,1,I didn't receive any new mail. I had checked.,I didn't receive any new mail. I had checked.,"[-0.010691414587199688, -0.01292553823441267, ...","[Nai yar mje to pta e nai kuch, Mjhe tu ni lgt..."
3,Yr us din mai pura din bzy rahe vo mujy awne h...,disgust,3,"That day, they were busy all day and not givin...",YR Us Din Mai Pura Din Bzy Rahe Vo Mujy Awne H...,Yr us din mai pura din bzy rahe vo mujy awne h...,Yr us din mai pura din bzy rahe vo mujy awne h...,negative,0,"I was busy the whole day on that day, they wer...","I was busy the whole day on that day, they wer...","[0.009936108253896236, -0.016926730051636696, ...",[Yr masla pta kia ha. Yad ha tm log us din upa...
4,Lakin wo abhe dar dar ka chalata ha,fear,7,But he still walks cautiously.,But it still moves at the rate,But Wu runs the cedar,Lakin wo abhe dar dar ka chalata ha,neutral,1,But he still walks with fear and hesitation.,But he still walks with fear and hesitation.,"[0.019262924790382385, -0.0011249196249991655,...","[Pata nahe ab kahan chala gea hai kamina, Han ..."


In [30]:
# light text cleaning (should I use clean regex for better accuracy?)
pad_punct    = re.compile('([^a-zA-Z ]+)')
multi_spaces = re.compile('\s{2,}')
#clean        = re.compile('[^a-zA-Z0-9,.?!\'\s]+')

def clean_text(s):
    s = s.replace('\n', ' ')
    s = pad_punct.sub(r' \1 ', s)
    #s = clean.sub(' ', s)
    s = multi_spaces.sub(' ', s)
    return s.strip()

df_train['text_clean'] = df_train['text'].apply( clean_text )
df_dev['text_clean']   = df_dev['text'].apply( clean_text )
df_test['text_clean']  = df_test['Text'].apply( clean_text )

In [31]:
# remove overlap with validation sets
val_sets = df_dev['text_clean'].tolist() + df_test['text_clean'].tolist()
print(len(val_sets), len(set(val_sets)))

print(df_train.shape)
df_train = df_train[ ~df_train['text_clean'].isin(val_sets) ]
print(df_train.shape)

2382 2206
(9530, 7)
(8151, 7)


In [32]:
# remove duplicates from train set
df_train = df_train.drop_duplicates(subset=['text_clean', 'emotion'])
print(df_train.shape)

(6167, 7)


# ChatGPT API: Few-Shot Classification

## Helper functions

In [19]:
openai.api_key = os.getenv("OPENAI_API_KEY")
model          = 'gpt-3.5-turbo'
labels_set     = { 'neutral', 'joy', 'trust', 'disgust', 'optimism', 'anticipation', 'sadness', 'fear',
                   'surprise', 'anger', 'pessimism', 'love', }
clean = re.compile(r'[^a-zA-Z ]+')
multi_spaces = re.compile('\s{2,}')
print(labels_set)

{'disgust', 'joy', 'anticipation', 'sadness', 'fear', 'surprise', 'optimism', 'pessimism', 'neutral', 'anger', 'trust', 'love'}


In [20]:
def verify_label(label_):
    '''
       Verify if label_ contains any of the categories
       from the predefined set of labels
    '''
    label_ = clean.sub(' ', label_)
    label_ = multi_spaces.sub(' ', label_).lower().split()
    res    = [i for i in label_ if i in labels_set]
    res    = list(set(res))
    return '/'.join(res) if res else None

In [21]:
def verify_num_tokens(model, messages):
    '''Check that there is enough tokens available for a ChatGPT repsonse'''
    num_tokens_tiktoken = num_tokens_from_messages(messages, model)
    if num_tokens_tiktoken > 4080:
        print(f'Number of tokens is {num_tokens_tiktoken} which exceeds 4080')        
        return False
    else:
        return True


@backoff.on_exception(backoff.expo, openai.error.RateLimitError, max_time=10)
def get_response(model, messages, temperature=0, max_tokens=None):
    '''Send request, return reponse'''
    response  = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = temperature,        # range(0,2), the more the less deterministic / focused
        top_p = 1,                        # top probability mass, e.g. 0.1 = only tokens from top 10% proba mass
        n = 1,                            # number of chat completions
        #max_tokens = max_tokens,          # tokens to return
        stream = False,        
        stop=None,                        # sequence to stop generation (new line, end of text, etc.)
        )
    content = response['choices'][0]['message']['content'].strip()
    #num_tokens_api = response['usage']['prompt_tokens']
    return content

## Approach 3: concatenate the closest few shot examples using the ChatGPT chat mode

In [42]:
model          = 'gpt-3.5-turbo'
embedding_type = 'gpt_embedding'
initial_prompt = 'Learn from the following examples of texts with assigned categories. ' +\
                 'Using this knowledge, select the most relevant category for the very last text below ' +\
                 'from the following list of categories: neutral, joy, trust, disgust, optimism, ' +\
                 'anticipation, sadness, fear, surprise, anger, pessimism, love. ' +\
                 'Output only one most relevant category for the very last text below.'
print(initial_prompt)

# Using followup questions improves the reponse. but ChatGPT can change its mind too easily sometimes
followup1 = 'Are you sure about that? If yes, output the same category, if no change the category'
print(followup1)

Learn from the following examples of texts with assigned categories. Using this knowledge, select the most relevant category for the very last text below from the following list of categories: neutral, joy, trust, disgust, optimism, anticipation, sadness, fear, surprise, anger, pessimism, love. Output only one most relevant category for the very last text below.
Are you sure about that? If yes, output the same category, if no change the category


In [43]:
# Using top_n closest embeddings, create ChatGPT messages object (alternating user (text)/assistant(category) Q&As)
def create_messages(df_, closest_texts, top_n=100 ):
    df_temp = df_[ df_['text'].isin(closest_texts[:top_n]) ]    
    text0, emotion0 = df_temp[['text', 'emotion']].values[0]
    messages = [ { "role": "system", "content": "You are a helpful text classifier.", },
                 { "role": "user", "content": initial_prompt + f' Text: {text0}', },
                 { "role": "assistant", "content": f'Category: {emotion0}', }
               ]    
    for text, emotion in df_temp[['text', 'emotion']].values[1:]:                  # emotion instead of target here
        messages += [
            { "role": "user", "content": f'Text: {text}', },
            { "role": "assistant", "content": f'Category: {emotion}', }
        ]
    while num_tokens_from_messages(messages) > 4000:
        messages = messages[:-2]
    return messages

In [44]:
def classify_text_few_shot3(text_, messages_, model):
    '''Classify text_ using prompt_ and ChatGPT API'''
    messages_ = deepcopy(messages_)
    messages_ += [
        { "role": "user", "content": f'Text: {text_}', },
    ]
    while num_tokens_from_messages(messages_) > 4080:
        messages_ = messages_[:-2] + [messages_[-1]]
    if not verify_num_tokens(model, messages_): return None
    label_    = get_response(model, messages_)
    old_label = label_
    label_    = verify_label(label_)        # get just the category if response is too long
        
    # if label not found in response text - second, extended chat
    if label_ is None:
        messages_ += [
            { "role": "assistant", "content": old_label, },
            { "role": "user", "content": followup1, }
            ]        
        label_    = get_response(model, messages_)
        old_label = label_
        label_    = verify_label(label_)        # get just the category if response is too long
            
    return label_ if label_ is not None else old_label

In [45]:
# how one messages object looks
c = df_dev['closest_texts'].values[3]
create_messages(df_train, c, top_n=10)

[{'role': 'system', 'content': 'You are a helpful text classifier.'},
 {'role': 'user',
  'content': 'Learn from the following examples of texts with assigned categories. Using this knowledge, select the most relevant category for the very last text below from the following list of categories: neutral, joy, trust, disgust, optimism, anticipation, sadness, fear, surprise, anger, pessimism, love. Output only one most relevant category for the very last text below. Text: Yr us k bahi bhr chala gaya na'},
 {'role': 'assistant', 'content': 'Category: neutral'},
 {'role': 'user',
  'content': 'Text: Yr masla pta kia ha. Yad ha tm log us din upar aie tha to papa bol raha tha gusa kr raha tha. To mja yaken kr us din papa pe bht gusa aya ka humari jaha bni hue ha waha to kam se kam rehna dae. Aksar esa hta ha mra ya arslan ka koe dost milnae a jae to papa dant ke bhej deta ha.'},
 {'role': 'assistant', 'content': 'Category: anger'},
 {'role': 'user',
  'content': 'Text: O ja yar tujy btya tu th

In [ ]:
# this simple iteration is faster than pandas df with tqdm
model = 'gpt-3.5-turbo'
start = time.time()
res   = dict()
count = 0
for t, closest in df_dev[['text', 'closest_texts']].values:
    if t in res:
        continue
    messages = create_messages(df_train, closest, top_n=100)
    try:
        res[ t ] = classify_text_few_shot3(t, messages, model)
    except openai.error.RateLimitError:
        print(f'\nText: {t}.\nRate limit error\n')
    except Exception as e:
        print(f'\nText: {t}\nError: {e}\n')
                
    count += 1    
    if count % 10 == 0:
        print(f'Processing text {count}. Time elapsed: {round((time.time()-start)/60, 4)} min')
        with open('data/res.pkl', 'wb') as f:
            pickle.dump(res, f, protocol=pickle.HIGHEST_PROTOCOL)
                        
        
elapsed = (time.time() - start)/60
print(f'\nTime elapsed {round(elapsed, 4)} min')
#file = 'data/res.pkl'
#with open(file, 'rb') as f:
#    res2 = pickle.load(handle)

Processing text 10. Time elapsed: 0.3015 min
Processing text 20. Time elapsed: 0.5234 min
Processing text 30. Time elapsed: 0.736 min
Processing text 40. Time elapsed: 0.9475 min
Processing text 50. Time elapsed: 1.1958 min
Processing text 60. Time elapsed: 1.4382 min
Processing text 70. Time elapsed: 1.6765 min
Processing text 80. Time elapsed: 2.4385 min
Processing text 90. Time elapsed: 2.6749 min
Processing text 100. Time elapsed: 2.9093 min
Processing text 110. Time elapsed: 3.1551 min
Processing text 120. Time elapsed: 3.3923 min
Processing text 130. Time elapsed: 3.6347 min
Processing text 140. Time elapsed: 3.8446 min
Processing text 150. Time elapsed: 4.1023 min
Processing text 160. Time elapsed: 4.319 min
Processing text 170. Time elapsed: 4.5679 min
Processing text 180. Time elapsed: 4.7884 min
Processing text 190. Time elapsed: 5.1195 min
Processing text 200. Time elapsed: 5.3379 min
Processing text 210. Time elapsed: 6.0717 min
Processing text 220. Time elapsed: 6.297 min


In [37]:
df_dev['gpt_pred'] = df_dev['text'].map( res )
print(df_dev.isna().sum())
df_dev['gpt_pred'].value_counts()

text                         0
emotion                      0
target                       0
gtp_translated               0
translated_hi                0
translated_ur                0
text_clean                   0
gpt_pred                     0
gpt_pred_num                 0
gpt_translated2              0
gpt_translated2_corrected    0
gpt_embedding                0
closest_texts                0
dtype: int64


neutral                                                                                                                                                                                                                                                                                             375
trust                                                                                                                                                                                                                                                                                               211
joy                                                                                                                                                                                                                                                                                                 130
anticipation                                                                                                    

In [63]:
df_dev['gpt_pred_target'] = df_dev['gpt_pred'].apply( lambda x: label2key.get(x) )
print(df_dev['gpt_pred_target'].isna().sum())
df_dev['gpt_pred_target'].value_counts()

1    848
0    343
Name: gpt_pred_binary, dtype: int64

In [64]:
y_dev      = df_dev['target'].values
y_dev_pred = df_dev['gpt_pred_target'].values
print( classification_report( y_dev, y_dev_pred, digits=4 ) )

              precision    recall  f1-score   support

           0     0.5948    0.5258    0.5581       388
           1     0.7830    0.8269    0.8044       803

    accuracy                         0.7288      1191
   macro avg     0.6889    0.6763    0.6813      1191
weighted avg     0.7217    0.7288    0.7241      1191



## Approach 2: concatenate random few shot examples using the ChatGPT chat mode

Split df into n random chunks to be used as few-shot examples:

In [126]:
# divide df into n chunks - this allows for 3k to 3.8k tokens per example
n          = 100
key2label = { 0: 'neutral', 1: 'emotional', }
examples  = []
chunks = [ df_train.sample(frac=1, random_state=random_state).iloc[i:i+n] for i in range(0, df_train.shape[0], n) ]
for ch in chunks:
    example = []
    for text, target in ch[['text', 'target']].values:
        example += [
            { "role": "user", "content": f'Text: {text}', },
            { "role": "assistant", "content": f'Category: {key2label[target]}', }
        ]
    examples.append(example)
        
lengths = [ num_tokens_from_messages(example) for example in examples ]
print(len(lengths))
print(lengths)

62
[3276, 3350, 3424, 3554, 3229, 3153, 3260, 3487, 3364, 3365, 3709, 3762, 3400, 3387, 3440, 3391, 3424, 3327, 3546, 3423, 3485, 3484, 3543, 3349, 3414, 3372, 3678, 3399, 3280, 3190, 3375, 3327, 3381, 3527, 3335, 3470, 3383, 3494, 3549, 3502, 3499, 3454, 3436, 3261, 3324, 3311, 3371, 3271, 3568, 3475, 3273, 3229, 3403, 3705, 3208, 3326, 3203, 3250, 3275, 3261, 3324, 2279]


In [127]:
# how one example looks
examples[2]

[{'role': 'user', 'content': 'Text: kiu yar a jana mza aye ga!?'},
 {'role': 'assistant', 'content': 'Category: emotional'},
 {'role': 'user', 'content': 'Text: Mujhe msg oe msg kr rhi he'},
 {'role': 'assistant', 'content': 'Category: neutral'},
 {'role': 'user',
  'content': 'Text: ma ab tk k exprnce k hwly sy bl rha hn. life to pta h bht h'},
 {'role': 'assistant', 'content': 'Category: neutral'},
 {'role': 'user', 'content': "Text: Mene jannat k pattay prh liya atlast :')"},
 {'role': 'assistant', 'content': 'Category: emotional'},
 {'role': 'user',
  'content': 'Text: Or wo achi khaasi intelligent t tbi itni okhi paheli boojh li t :P'},
 {'role': 'assistant', 'content': 'Category: emotional'},
 {'role': 'user',
  'content': 'Text: Tm logo  ka off ho gaixa hn aur tm ne rida ki pic daekane the . ...,'},
 {'role': 'assistant', 'content': 'Category: neutral'},
 {'role': 'user',
  'content': 'Text: Pehla name mera e c jinna nu pta a,par mainu ki pta a, ae mainu nai pts'},
 {'role': 'as

In [128]:
def classify_text_few_shot2(text_, messages_, model):
    '''Classify text_ using prompt_ and ChatGPT API'''
    messages_ = deepcopy(messages_)
    messages_ += [
        { "role": "user", "content": f'Text: {text_}', },
    ]
    if not verify_num_tokens(model, messages_): return None
    label_    = get_response(model, messages_)
    old_label = label_
    label_    = verify_label(label_)        # get just the category if response is too long
        
    # if label not found in response text - second, extended chat
    if label_ is None:
        messages_ += [
            { "role": "assistant", "content": old_label, },
            { "role": "user", "content": followup1, }
            ]        
        label_    = get_response(model, messages_)
        old_label = label_
        label_    = verify_label(label_)        # get just the category if response is too long
            
    return label_ if label_ is not None else old_label

In [129]:
start  = time.time()
res    = dict()
count1 = 0
count2 = 0
for t in df_dev['text'].tolist():
    if t in res:
        continue
    if count2 >= len(examples):
        count2 = 0
    messages = examples[ count2 ]
    count2 += 1
    try:
        res[ t ] = classify_text_few_shot2(t, messages, model)
    except openai.error.RateLimitError:
        print(f'\nText: {t}.\nRate limit error\n')
    except Exception as e:
        print(f'\nText: {t}\nError: {e}\n')
                
    count1 += 1    
    if count1 % 10 == 0:
        print(f'Processing text {count1}; example {count2-1}. Time elapsed: {round((time.time()-start)/60, 4)} min')
        with open('data/res.pkl', 'wb') as f:
            pickle.dump(res, f, protocol=pickle.HIGHEST_PROTOCOL)
                        
        
elapsed = (time.time() - start)/60
print(f'\nTime elapsed {round(elapsed, 4)} min')
#file = 'data/res.pkl'
#with open(file, 'rb') as f:
#    res2 = pickle.load(handle)

Processing text 10; example 9. Time elapsed: 0.2034 min
Processing text 20; example 19. Time elapsed: 0.3919 min
Processing text 30; example 29. Time elapsed: 0.5834 min
Processing text 40; example 39. Time elapsed: 0.7687 min
Processing text 50; example 49. Time elapsed: 0.9588 min
Processing text 60; example 59. Time elapsed: 1.1971 min
Processing text 70; example 7. Time elapsed: 1.409 min
Processing text 80; example 17. Time elapsed: 1.6186 min
Processing text 90; example 27. Time elapsed: 1.8049 min
Processing text 100; example 37. Time elapsed: 2.0017 min
Processing text 110; example 47. Time elapsed: 2.1894 min
Processing text 120; example 57. Time elapsed: 2.3815 min
Processing text 130; example 5. Time elapsed: 2.5695 min
Processing text 140; example 15. Time elapsed: 2.7638 min
Processing text 150; example 25. Time elapsed: 2.9498 min
Processing text 160; example 35. Time elapsed: 3.1631 min
Processing text 170; example 45. Time elapsed: 3.4141 min
Processing text 180; exampl

In [130]:
df_dev['gpt_pred'] = df_dev['text'].map( res )
df_dev['gpt_pred'].value_counts()

emotional                                       808
neutral                                         382
Yes, I'm sure. The category is motivational.      1
Name: gpt_pred, dtype: int64

In [131]:
df_dev['gpt_pred_binary'] = df_dev['gpt_pred'].apply( lambda x: 0 if x=='neutral' else 1 )
df_dev['gpt_pred_binary'].value_counts()

1    809
0    382
Name: gpt_pred_binary, dtype: int64

In [132]:
y_dev      = df_dev['target'].values
y_dev_pred = df_dev['gpt_pred_binary'].values
print( classification_report( y_dev, y_dev_pred, digits=4 ) )

              precision    recall  f1-score   support

           0     0.4895    0.4820    0.4857       388
           1     0.7515    0.7572    0.7543       803

    accuracy                         0.6675      1191
   macro avg     0.6205    0.6196    0.6200      1191
weighted avg     0.6662    0.6675    0.6668      1191



## Binary approach 1: concatenate random few-shot examples into one long string

Example prompt:
```
Message: Support has been terrible for 2 weeks...
Sentiment: Negative
###
Message: I love your API, it is simple and so fast!
Sentiment: Positive
###
Message: GPT-J has been released 2 months ago.
Sentiment: Neutral
###
Message: The reactivity of your team has been amazing, thanks!
Sentiment:
```

In [25]:
# total tokens in training set
messages = [{ 'role': 'user', 'content': ' '.join(df_train['text_clean'].tolist()) }]
total_tokens        = num_tokens_from_messages(messages)
avg_tokens_per_msg  = total_tokens / df_train.shape[0]
msg_per_window_size = 4096 /  avg_tokens_per_msg
print('Total tokens:', total_tokens)
print('Avg tokens per msg:', avg_tokens_per_msg)
print('Avg messages per window size:', msg_per_window_size)

Total tokens: 116049
Avg tokens per msg: 18.817739581644236
Avg messages per window size: 217.66695102930657


Split df into n random chunks to be used as few-shot examples:

In [91]:
# divide df into n chunks - this allows for 3k to 3.8k tokens per example
n         = 135
key2label = { 0: 'neutral', 1: 'emotional', }
examples  = []
chunks  = [ df_train.sample(frac=1, random_state=random_state).iloc[i:i+n] for i in range(0, df_train.shape[0], n) ]
for ch in chunks:
    example = ''
    for text, target in ch[['text', 'target']].values:
        example += f'Message: {text}.\nCategory: {key2label[target]}.\n####\n'
    examples.append(example)
        
lengths = [ num_tokens_from_messages([{'role':'user', 'content': example} ]) for example in examples ]
print(lengths)

[3472, 3690, 3662, 3346, 3394, 3608, 3630, 3805, 4022, 3726, 3590, 3530, 3729, 3633, 3722, 3633, 3795, 3630, 3613, 3808, 3540, 3380, 3597, 3528, 3697, 3586, 3634, 3586, 3825, 3750, 3782, 3620, 3472, 3519, 3542, 3583, 3794, 3416, 3455, 3977, 3427, 3476, 3302, 3400, 3531, 2424]


In [105]:
len(lengths)

46

In [92]:
print(examples[0])

Message: Yr jtni jldi ml jaye utna acha he....
Category: emotional.
####
Message: Haha mei kahan se sweet hun :p.
Category: emotional.
####
Message: Han abi kuch theek hy shop pr ly kr gya tha.
Category: neutral.
####
Message: I think .. he made his own slides....
Category: emotional.
####
Message: Plz give me bike keys. I m in office.
Category: emotional.
####
Message: Teri do Aakhiyaan Wich Dubb Mrya,.
Category: emotional.
####
Message: Haha ni wo boy ni smjy g q k wo khud boy ho g.
Category: emotional.
####
Message: Mera Sirf DataStructure Ka Pta Chala Hai. & Your?.
Category: emotional.
####
Message: Salam Faiza Baaji
How are you  ... 
Plz take a  good care of urself. 
Aur jo treat meri taraf due hai uska kiya karna hai?  :-).
Category: emotional.
####
Message: Sir transport dept walay recomendation letter mang rahay han....
Category: neutral.
####
Message: Yar  Maine fee a hi tak Jamaica nahi krai 2000 fine ho gear hai to fine maaf kra sakta hai.
Category: neutral.
####
Message: Ok

In [93]:
def classify_text_few_shot(prompt_):
    '''Classify text_ using prompt_ and ChatGPT API'''
        
    # compose messages and check num_tokens
    messages = [
            #{ "role": "system", "content": "You are a very accurate text classifier.", },
            { "role": "user", "content": prompt_, },
            ]
    if not verify_num_tokens(model, messages): return None
    label_    = get_response(model, messages)
    old_label = label_
    label_    = verify_label(label_)        # get just the category if response is too long
        
    # if label not found in response text - second, extended chat
    if label_ is None:
        messages += [
            { "role": "assistant", "content": old_label, },
            { "role": "user", "content": followup1, }
            ]        
        label_    = get_response(model, messages)        
        old_label = label_
        label_    = verify_label(label_)        # get just the category if response is too long
            
    return label_ if label_ is not None else old_label

In [100]:
start  = time.time()
#res    = dict()
count1 = 0
count2 = 0
for t in df_dev['text'].tolist():
    if t in res:
        continue
    if count2 >= len(examples):
        count2 = 0
    prompt = examples[ count2 ] + f'\nMessage: {t}.\nCategory:'
    count2 += 1
    try:
        res[ t ] = classify_text_few_shot(prompt)
    except openai.error.RateLimitError:
        print(f'\nText: {t}.\nRate limit error\n')
    except Exception as e:
        print(f'\nText: {t}\nError: {e}\n')
                
    count1 += 1    
    if count1 % 10 == 0:
        print(f'Processing text {count1}; example {count2-1}')
        with open('data/res.pkl', 'wb') as f:
            pickle.dump(res, f, protocol=pickle.HIGHEST_PROTOCOL)
                        
        
elapsed = (time.time() - start)/60
print(f'\nTime elapsed {round(elapsed, 4)} min')
#file = 'data/res.pkl'
#with open(file, 'rb') as f:
#    res2 = pickle.load(handle)


Time elapsed 0.1439 min


In [102]:
df_dev['gpt_pred'] = df_dev['text'].map( res )
df_dev['gpt_pred'].value_counts()

emotional                                        715
neutral                                          470
Yes, I'm sure. The category is motivational.       1
Yes, I'm sure. The category is offensive.          1
Yes, I'm sure. The category is inappropriate.      1
Yes, the category is offensive.                    1
Yes, I'm sure. The category is positive.           1
Yes, I'm sure. The category is humorous.           1
Name: gpt_pred, dtype: int64

In [103]:
df_dev['gpt_pred_binary'] = df_dev['gpt_pred'].apply( lambda x: 0 if x=='neutral' else 1 )
df_dev['gpt_pred_binary'].value_counts()

1    721
0    470
Name: gpt_pred_binary, dtype: int64

In [104]:
y_dev      = df_dev['target'].values
y_dev_pred = df_dev['gpt_pred_binary'].values
print( classification_report( y_dev, y_dev_pred, digits=4 ) )

              precision    recall  f1-score   support

           0     0.4723    0.5722    0.5175       388
           1     0.7698    0.6912    0.7283       803

    accuracy                         0.6524      1191
   macro avg     0.6211    0.6317    0.6229      1191
weighted avg     0.6729    0.6524    0.6597      1191



## APPENDIX

### Prompts and results

__Experiment 1__  
Split the training set into random non-stratified 62 chunks, each 100 examples long (this allows not to exceed the ChatGPT window size of 4096 tokens). Convert the 100 examples in each chunk into the messages object containing alternating user(text) and assistant (category) chat utterances. Iterate over chunks and dev set single examples and for each dev set example use the next message; the dev sety example is added as the next user utterance for few-shot learning - ChatGPT to respond with an assitant utterance containing the category.
```

```
_____